In [2]:
import numpy as np
import cv2
import pandas as pd
from scipy import ndimage as nd
 
import pickle
import os
from skimage.io import imread, imshow
from skimage.filters import roberts, sobel, scharr, prewitt
import glob

import tensorflow as tf
import random
from skimage.io import imread, imshow
import skimage.transform
import matplotlib.pyplot as plt

### Prepare the Dataset

In [44]:
source_paths1 = "S1Hand/"
source_paths2 = "S2Hand"
label_path = "LabelHand/"

sources1 = []
sources2 =[]
label = []

for file in os.listdir(source_paths1):
    if file.endswith("tif"):
      sources1.append(file)

for file in os.listdir(source_paths2):
    if file.endswith("tif"):
      sources2.append(file)
        
for file in os.listdir(label_path):
    if file.endswith("tif"):
      label.append(file)


## SAnity Check
sources1 = sorted(sources1)
sources2 = sorted(sources2)
label = sorted(label)
print(len(sources1))
print(len(sources2))
print(len(label))

577
577
577


In [45]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 7


#Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [46]:
X = np.zeros((len(sources1), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y = np.zeros((len(label),IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)

In [47]:
# Sanity Check
print(X.shape)
print(Y.shape)

(577, 128, 128, 7)
(577, 128, 128, 1)


In [48]:
for count, file in enumerate(sources1):
    source_img = imread(source_paths1 +'/' +file ) #[0:2,:,:]
    name = file.split("_")
    s2_name = name[0] + "_" +  name[1] + "_" + "S2Hand_" + name[3]

    s1_image = np.transpose(source_img, (1, 2, 0))
    s2_image =imread(source_paths2+'/' +s2_name)
    s2_image = np.transpose(s2_image, (1, 2, 0))

    merged = np.concatenate((s2_image, s1_image), axis=2)
    X[count] = merged


for count, file in enumerate(label):
    slabel_img = imread(label_path + '/'+file ) #[:,:,:,IMG_CHANNELS]
    # print(slabel_img.min())
    y = np.expand_dims(slabel_img, axis=2)
    # print(f"The shape of y labels: {y.min()}")
    Y[count] = y 

In [49]:
from sklearn.preprocessing import LabelEncoder 
labelencoder = LabelEncoder()
Y_resahped = Y.reshape(-1,1)
Y_resahped_encoded = labelencoder.fit_transform(Y_resahped)
Y_original = Y_resahped_encoded.reshape(577,128,128,1)
print(np.unique(Y_original))

[0 1 2]


### Define the UNET

In [50]:
#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(3, (1, 1), activation='softmax')(c9) #sigmoid
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 7  0           []                               
                                )]                                                                
                                                                                                  
 lambda_1 (Lambda)              (None, 128, 128, 7)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 128, 128, 16  1024        ['lambda_1[0][0]']               
                                )                                                                 
                                                                                            

### Train the Model

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y_original, test_size=0.10)

In [52]:
from keras.utils import to_categorical
train_mask_cat = to_categorical(y_train, num_classes=3)
y_train_cat = train_mask_cat.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2],3)
print(y_train_cat.shape)

(519, 128, 128, 3)


In [53]:
test_mask_cat = to_categorical(y_test, num_classes=3)
y_test_cat = test_mask_cat.reshape(y_test.shape[0], y_test.shape[1], y_test.shape[2], 3)
print(y_test_cat.shape)

(58, 128, 128, 3)


In [54]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_S1S2.h5', verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, monitor='val_loss'),checkpointer]

results = model.fit(x_train, y_train_cat, validation_split=0.1, batch_size=12, epochs=150, callbacks=callbacks)

Epoch 1/150
38/39 [============================>.] - ETA: 0s - loss: 0.9866 - accuracy: 0.5342
Epoch 1: val_loss improved from inf to 0.95362, saving model to model_S1S2.h5
39/39 [==============================] - 4s 67ms/step - loss: 0.9847 - accuracy: 0.5367 - val_loss: 0.9536 - val_accuracy: 0.5800
Epoch 2/150
39/39 [==============================] - ETA: 0s - loss: 0.9226 - accuracy: 0.5887
Epoch 2: val_loss improved from 0.95362 to 0.93723, saving model to model_S1S2.h5
39/39 [==============================] - 2s 41ms/step - loss: 0.9226 - accuracy: 0.5887 - val_loss: 0.9372 - val_accuracy: 0.5805
Epoch 3/150
39/39 [==============================] - ETA: 0s - loss: 0.8916 - accuracy: 0.5959
Epoch 3: val_loss improved from 0.93723 to 0.87214, saving model to model_S1S2.h5
39/39 [==============================] - 2s 41ms/step - loss: 0.8916 - accuracy: 0.5959 - val_loss: 0.8721 - val_accuracy: 0.6110
Epoch 4/150
39/39 [==============================] - ETA: 0s - loss: 0.8803 - accur

### Evaluate the model on the test data

In [55]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test_cat, batch_size=12)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 19ms/step - loss: 0.7304 - accuracy: 0.6656
test loss, test acc: [0.730411946773529, 0.6656094193458557]


In [56]:
# IoU
y_pred = model.predict(x_test)
y_pred_argmax = np.argmax(y_pred,  axis=3)

from keras.metrics import MeanIoU
n_classes = 3
IoU_Keras = MeanIoU(num_classes=n_classes)
IoU_Keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU: ", IoU_Keras.result().numpy())

2/2 [==============================] - 0s 33ms/step
Mean IoU:  0.3427987
